In [9]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))
metrics = MD.MetricsCallback()

def make_trainer():
  return L.Trainer(
    max_epochs=15,
    max_time={"minutes": 10},
    callbacks=[
      # metrics,
    ],
    deterministic=True,
    default_root_dir=here("cache/lightning"),
    enable_checkpointing=True,
    fast_dev_run=False,
    accelerator="auto", 
  )

trainer = make_trainer()
trainer

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# Distil Bert finetuned (cross entropy loss)

## 3 classes

In [ ]:
reload(MD)
distil_3 = MD.DistilBertFinetune(n_emotions=3)
if True:    
  trainer.fit(
    distil_3,
    datamodule=data_module_3,
  )
else:
  distil_3 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_3_14epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_3,
  datamodule=data_module_3,
)
del distil_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_work

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.48068591952323914
   test_expected_ndcg       0.6358739137649536
   test_f1_interesting       0.529083788394928
    test_f1_standard        0.5880631804466248
        test_ndcg           0.9344432353973389
        test_rmse           0.2569975256919861
    test_weighted_mse       0.12027940899133682
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.48068591952323914,
  'test_rmse': 0.2569975256919861,
  'test_ndcg': 0.9344432353973389,
  'test_expected_ndcg': 0.6358739137649536,
  'test_f1_standard': 0.5880631804466248,
  'test_f1_interesting': 0.529083788394928,
  'test_weighted_mse': 0.12027940899133682}]

## 7 classes

In [ ]:
reload(MD)
distil_7 = MD.DistilBertFinetune(n_emotions=7)
if True:    
  trainer.fit(
    distil_7,
    data_module=data_module_7,
  )
else:
  distil_7 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_7_14epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_7,
  datamodule=data_module_7
)
del distil_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.28261885046958923
   test_expected_ndcg       0.41945603489875793
   test_f1_interesting      0.2747745215892792
    test_f1_standard         0.220706507563591
        test_ndcg           0.8937562704086304
        test_rmse           0.18835683166980743
    test_weighted_mse       0.06440073251724243
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.28261885046958923,
  'test_rmse': 0.18835683166980743,
  'test_ndcg': 0.8937562704086304,
  'test_expected_ndcg': 0.41945603489875793,
  'test_f1_standard': 0.220706507563591,
  'test_f1_interesting': 0.2747745215892792,
  'test_weighted_mse': 0.06440073251724243}]

## 28 classes

In [ ]:
reload(MD)
distil_28 = MD.DistilBertFinetune(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_28,
    datamodule=data_module_28,
  )
else:
  distil_28 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_28_11epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.11774969100952148
   test_expected_ndcg       0.27364346385002136
   test_f1_interesting      0.09371107816696167
    test_f1_standard        0.08957261592149734
        test_ndcg            0.811306357383728
        test_rmse           0.11224716156721115
    test_weighted_mse      0.022755902260541916
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.11774969100952148,
  'test_rmse': 0.11224716156721115,
  'test_ndcg': 0.811306357383728,
  'test_expected_ndcg': 0.27364346385002136,
  'test_f1_standard': 0.08957261592149734,
  'test_f1_interesting': 0.09371107816696167,
  'test_weighted_mse': 0.022755902260541916}]

In [82]:
# test sample
trainer.predict(
  distil_28,
  ["I Love this"],
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[3.5273e-04, 1.1074e-02, 9.5317e-01, 3.4014e-03, 1.6734e-04, 5.0980e-04,
          3.6321e-04, 7.9082e-05, 1.3321e-01, 1.1438e-03, 2.9806e-04, 4.7053e-05,
          1.0698e-03, 8.4968e-05, 1.1731e-02, 3.5829e-03, 5.2930e-02, 1.5770e-05,
          1.8953e-02, 3.7406e-05, 3.8242e-06, 3.3380e-04, 1.1921e-04, 1.4612e-05,
          1.2453e-03, 2.8391e-03, 2.5755e-04, 6.7899e-06]])]

In [83]:
del distil_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')

# Distil Bert finetuned (DCG loss)

## 3 classes

In [ ]:
reload(MD) # DistilBertFinetuneOnDCG
distil_dcg_3 = MD.DistilBertFinetuneOnDCG(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_3,
    datamodule=data_module_3,
  )
else:
  distil_dcg_3 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_3_10epchs.ckpt"),
    n_emotions=3,
  ) 
out = trainer.test(
  distil_dcg_3,
  datamodule=data_module_3,
)
del distil_dcg_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.5870808959007263
   test_expected_ndcg       0.6324573755264282
   test_f1_interesting      0.4523560702800751
    test_f1_standard        0.7240345478057861
        test_ndcg           0.9325181841850281
        test_rmse           0.3293803930282593
    test_weighted_mse       0.15471822023391724
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.5870808959007263,
  'test_rmse': 0.3293803930282593,
  'test_ndcg': 0.9325181841850281,
  'test_expected_ndcg': 0.6324573755264282,
  'test_f1_standard': 0.7240345478057861,
  'test_f1_interesting': 0.4523560702800751,
  'test_weighted_mse': 0.15471822023391724}]

## 7 classes

In [ ]:
reload(MD)
distil_dcg_7 = MD.DistilBertFinetuneOnDCG(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_7,
    datamodule=data_module_7,
  )
else:
  distil_dcg_7 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_7_10epchs.ckpt"),
    n_emotions=7,
  )
  
out = trainer.test(
  distil_dcg_7,
  datamodule=data_module_7,
)
del distil_dcg_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4218612313270569
   test_expected_ndcg       0.4180324375629425
   test_f1_interesting      0.18402419984340668
    test_f1_standard        0.3870481550693512
        test_ndcg           0.8868971467018127
        test_rmse            0.286935031414032
    test_weighted_mse       0.10041247308254242
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4218612313270569,
  'test_rmse': 0.286935031414032,
  'test_ndcg': 0.8868971467018127,
  'test_expected_ndcg': 0.4180324375629425,
  'test_f1_standard': 0.3870481550693512,
  'test_f1_interesting': 0.18402419984340668,
  'test_weighted_mse': 0.10041247308254242}]

## 28 classes

In [ ]:
reload(MD)
distil_dcg_28 = MD.DistilBertFinetuneOnDCG(n_emotions=28)
trainer = make_trainer()
if True:
  trainer.fit(
    distil_dcg_28,
    datamodule=data_module_28,
  )
else:
  distil_dcg_28 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_dcg_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4454554617404938
   test_expected_ndcg       0.2676355242729187
   test_f1_interesting      0.05491499975323677
    test_f1_standard        0.35522475838661194
        test_ndcg            0.790400505065918
        test_rmse           0.34306931495666504
    test_weighted_mse       0.12370707094669342
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4454554617404938,
  'test_rmse': 0.34306931495666504,
  'test_ndcg': 0.790400505065918,
  'test_expected_ndcg': 0.2676355242729187,
  'test_f1_standard': 0.35522475838661194,
  'test_f1_interesting': 0.05491499975323677,
  'test_weighted_mse': 0.12370707094669342}]

In [87]:
# test sample
trainer.predict(
  distil_dcg_28,
  ["I Love this"],
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[0.0433, 0.6453, 0.9952, 0.3850, 0.0246, 0.2627, 0.0545, 0.0138, 0.9210,
          0.1429, 0.0670, 0.0282, 0.2872, 0.1308, 0.5913, 0.3643, 0.8826, 0.0120,
          0.6599, 0.0147, 0.0060, 0.0908, 0.0364, 0.0083, 0.1318, 0.5724, 0.0764,
          0.0056]])]

In [88]:
del distil_dcg_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')

# Distil Bert finetuned (Weighed MSE loss)

## 3 classes

In [89]:
reload(MD)
distil_mse_3 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_3,
    datamodule=data_module_3,
  )
else:
  distil_mse_3 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_3_10epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_mse_3,
  datamodule=data_module_3,
)
del distil_mse_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss       | BCEWith

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.47133371233940125
   test_expected_ndcg       0.6306819319725037
   test_f1_interesting      0.5256281495094299
    test_f1_standard        0.6842607259750366
        test_ndcg           0.9430682063102722
        test_rmse           0.25222402811050415
    test_weighted_mse       0.09928575903177261
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.47133371233940125,
  'test_rmse': 0.25222402811050415,
  'test_ndcg': 0.9430682063102722,
  'test_expected_ndcg': 0.6306819319725037,
  'test_f1_standard': 0.6842607259750366,
  'test_f1_interesting': 0.5256281495094299,
  'test_weighted_mse': 0.09928575903177261}]

## 7 classes

In [90]:
reload(MD)
distil_mse_7 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_7,
    datamodule=data_module_7,
  )
else:
  distil_mse_7 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_7_10epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_mse_7,
  datamodule=data_module_7,
)
del distil_mse_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss       | BCEWith

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.2903909981250763
   test_expected_ndcg       0.41924792528152466
   test_f1_interesting      0.2989925742149353
    test_f1_standard        0.3715859055519104
        test_ndcg           0.9057422280311584
        test_rmse           0.1902606040239334
    test_weighted_mse       0.05312378704547882
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.2903909981250763,
  'test_rmse': 0.1902606040239334,
  'test_ndcg': 0.9057422280311584,
  'test_expected_ndcg': 0.41924792528152466,
  'test_f1_standard': 0.3715859055519104,
  'test_f1_interesting': 0.2989925742149353,
  'test_weighted_mse': 0.05312378704547882}]

## 28 classes

In [3]:
reload(MD)
distil_mse_28 = MD.DistilBertFinetuneOnWeightedMSE(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_mse_28,
    datamodule=data_module_28,
  )
else:
  distil_mse_28 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onWeightedMSE_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_mse_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                                | Params | Mode 
-------------------------------------------------------------------------------
0 | model          | DistilBertForSequenceClassification | 67.0 M | eval 
1 | sigmoid        | Sigmoid                             | 0      | train
2 | sig_loss       | BCEWith

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.11979646235704422
   test_expected_ndcg       0.2729625105857849
   test_f1_interesting      0.1449052095413208
    test_f1_standard        0.1816653460264206
        test_ndcg           0.8243889808654785
        test_rmse           0.11022453010082245
    test_weighted_mse       0.01850876770913601
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.11979646235704422,
  'test_rmse': 0.11022453010082245,
  'test_ndcg': 0.8243889808654785,
  'test_expected_ndcg': 0.2729625105857849,
  'test_f1_standard': 0.1816653460264206,
  'test_f1_interesting': 0.1449052095413208,
  'test_weighted_mse': 0.01850876770913601}]

In [4]:
# test sample
trainer.predict(
  distil_mse_28,
  ["I Love this"],
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[6.2583e-03, 8.6624e-03, 9.7745e-01, 9.6571e-04, 1.4675e-03, 4.7952e-04,
          1.2525e-03, 4.3724e-04, 2.7103e-01, 4.4094e-03, 4.5573e-04, 4.2293e-04,
          7.6382e-03, 7.5037e-04, 1.5402e-02, 2.2507e-03, 3.3154e-02, 3.7778e-04,
          2.0479e-02, 3.9696e-04, 7.5781e-05, 4.6610e-04, 1.6086e-03, 3.1818e-04,
          2.3903e-03, 2.6991e-02, 6.0228e-05, 2.4498e-05]])]

In [5]:
del distil_mse_28
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')